因为我们保存的是参数，然后用的又是 ipynb，这里网络架构只好重新再写一下

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import mnist
from torchvision import transforms

In [2]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1=nn.Conv2d(1,6,5,padding=2)
        self.conv2=nn.Conv2d(6,16,5)
        self.pool=nn.AvgPool2d(2,2)
        self.fc1=nn.Linear(400,120)
        self.fc2=nn.Linear(120,84)
        self.out=nn.Linear(84,1)
    def forward(self,x):
        x=self.pool(F.relu(self.conv1(x)))
        x=self.pool(F.relu(self.conv2(x)))
        x=x.view(-1,self.num_flat_features(x))
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.out(x)
        return x
    def num_flat_features(self, x):
        size=x.size()[1:]
        num_features=1
        for s in size:
            num_features*=s
        return num_features
cnn=CNN()
cnn

CNN(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (out): Linear(in_features=84, out_features=1, bias=True)
)

In [3]:
state_dict=torch.load('saves/official_cnn_params.pth')
cnn.load_state_dict(state_dict)

<All keys matched successfully>

也得换一下保存的内存设备为gpu

In [4]:
device=torch.device('cuda:0')
cnn=cnn.to(device)

评估的时候，我一边挂着游戏，一边跑
结果提示 DataLoader worker(pid xxx) exited unexpectedly
线程数不够了哈哈，索性这里直接就不要 worker 了
主要还是训练费时间，从磁盘\cache取数也就那样吧

In [5]:
trans=transforms.Compose([transforms.ToTensor(),
                          transforms.Normalize([0.5],[0.5])]
)
train_set=mnist.MNIST('official_data',train=True,transform=trans)
test_set=mnist.MNIST('official_data',train=False,transform=trans)
train_loader=DataLoader(train_set,batch_size=64,shuffle=True)
test_loader=DataLoader(test_set,batch_size=64,shuffle=True)

In [18]:
x=train_loader.dataset[0][0]
x=x.view(-1,x.size()[0],x.size()[1],x.size()[2])
x=x.to(device)
y_pred=cnn(x)
y_pred

tensor([[5.0036]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [15]:
y_pred=y_pred.long() # 其实不建议这样写，tensor有long()却没有float32()就离谱，不统一
y_pred

tensor([[5]], device='cuda:0')

可以看到，OK的，没问题
接下来我们做一下评估
这里我们使用 torchmetrics
可以用 torchmetrics.functional.accuracy(preds,target)
也可以用 torchmetrics.Accuracy() 对象
这个对象你可以一批一批预测 batch，返回当前batch的acc，同时它内部还会累加样本数和正确数
对 batch 预测的同时，最后你可以 .compute() 得到整个样本的预测准确率
当你不需要累加时，只需 .reset() 即可
主要 train_loader 第一维是样本数，第二维是 (X,y) 的一个 tuple，不好直接取出整个特征集和标签集
如果这俩维度颠倒过来就方便了，可惜它不是这样
所以我会选用 Accuracy() 对象

In [23]:
import torchmetrics
train_accuracy=torchmetrics.Accuracy().to(device) # 似乎只要是对象就得挪一下，这还真是麻烦
test_accuracy=torchmetrics.Accuracy().to(device)
for X_train,y_train in iter(train_loader):
    X_train=X_train.to(device)
    y_train=y_train.to(device)
    y_train=y_train.reshape(y_train.size()[0],1)
    y_pred=cnn(X_train).to(torch.int)
    train_accuracy(y_pred,y_train)
for X_test,y_test in iter(test_loader):
    X_test=X_test.to(device)
    y_test=y_test.to(device)
    y_test=y_test.reshape(y_test.size()[0],1)
    y_pred=cnn(X_test).to(torch.int)
    test_accuracy(y_pred,y_test)
train_acc=train_accuracy.compute()
train_accuracy.reset()
test_acc=test_accuracy.compute()
test_accuracy.reset()
train_acc,test_acc

(tensor(0.2638, device='cuda:0'), tensor(0.2633, device='cuda:0'))

这个准确率着实真不咋地
主要一方面模型没怎么优化吧
另一方面训练的代数还是不够
网上的话优化版差不多100次才行，500次就很不错了
均方差5到10+，开根看的话基本就是一个差2，这误差确实太大了，就好比 4 平均可能预测到 2 到 6 这样

那么接下来，直接做一下 kaggle 的 MNIST，练一下 torch 对 csv 怎么使用的
顺便也优化一下咱们的cnn网络